# S3. Fine-tuning de modelo pre-entrenado para la clasificación de CIFAR-10
En esta sesión veremos como realizar el fine-tuning (ajuste fino) de una arquitectura de red [Inception](https://paperswithcode.com/paper/rethinking-the-inception-architecture-for), más concretamente veremos su [versión 3 (V3) disponible en Keras](https://keras.io/api/applications/inceptionv3/) pre-entrenada con la base de datos de imágenes [ImageNet](https://www.image-net.org) para la clasificación de CIFAR-10.

### Modelo preentrenado: Inception

La red Inception de Google (2015) basado en CNNs fue uno de los primeros modelos pre-entrenados pensados para que fuera aplicado a otras tareas (*transfer learning*) tras un ajuste fino del modelo. Estos modelos pre-entrenados presentan la arquitectura típica **stem-body-head:**
* **Stem (raíz):** $\;$ dos o tres capas convolucionales que extraen características de bajo nivel
* **Body (cuerpo):** $\;$ subred de **bloques convolucionales** repetidos
* **Head (cabeza):** $\;$ transforma la salida del cuerpo mediante un red densa según la tarea a abordar (clasificación, segmentación, etc.)

<div align="center">
<table><tr>
<td style="border: none;"><img src="https://github.com/CarlosIvars/APR/blob/main/Parte2/prac2/s1/P3/Figure_14.19.png?raw=1"/></td>
</tr></table>
</div>

Esta red toma su nombre de los bloque Inception que se definen en su cuerpo. En cada bloque Inception se concatenan ramas paralelas con kernels diferentes, para que cada capa del cuerpo escoja la que más le convenga
<div align="center">
<table><tr>
<td style="border: none;"><img src="https://github.com/CarlosIvars/APR/blob/main/Parte2/prac2/s1/P3/Figure_14.18.png?raw=1" width=600/></td>
</tr></table>
</div>

### Carga de datos
La utilización de una red pre-entrenada conlleva preprocesar nuestros datos con el mismo preproceso que se empleó para los datos de entrenamiento de la red pre-entrenada. En el caso de la red Inception V3, las imágenes necesitan, entre otras cosas, ser redimensionadas a 299 x 299. Si realizamos este preproceso para nuestro conjunto de entrenamiento, esto requeriría aproximadamente 40GB, por lo que es conveniente realizar este preproceso bajo demanda. Es decir, aplicaremos el preproceso para cada batch en el momento que vaya a ser utilizado.      

La aplicación de este preproceso bajo demanda está ya implementado en el módulo [tensorflow_datasets](https://www.tensorflow.org/datasets), así que en esta sesión cargaremos y manipularemos CIFAR-10 como un objeto [Dataset](https://www.tensorflow.org/api_docs/python/tf/data/Dataset) utilizando este módulo.

In [10]:
import tensorflow as tf
import tensorflow_datasets as tfds
train_data, test_data = tfds.load('caltech101', split=['train', 'test'], as_supervised=True)
train_size = len(train_data)

Dl Completed...: 0 url [00:00, ? url/s]

Dl Size...: 0 MiB [00:00, ? MiB/s]

Extraction completed...: 0 file [00:00, ? file/s]

Generating splits...:   0%|          | 0/2 [00:00<?, ? splits/s]

Generating train examples...:   0%|          | 0/3060 [00:00<?, ? examples/s]

Shuffling /root/tensorflow_datasets/caltech101/3.0.1.incompleteZLAOCB/caltech101-train.tfrecord*...:   0%|    …

Generating test examples...:   0%|          | 0/6084 [00:00<?, ? examples/s]

Shuffling /root/tensorflow_datasets/caltech101/3.0.1.incompleteZLAOCB/caltech101-test.tfrecord*...:   0%|     …

Dataset caltech101 downloaded and prepared to /root/tensorflow_datasets/caltech101/3.0.1. Subsequent calls will reuse this data.


### Preproceso

Primero definimos una función que aplica el preproceso necesario a una muestra de entrenamiento (imagen, etiqueta de clase). Esto incluye redimensionar la imagen a 299 x 299, el preproceso específico de la red InceptionV3 y convertir la etiqueta de clase a one-hot encoding.

In [ ]:
from keras.applications.inception_v3 import preprocess_input

img_size = (299, 299)
num_classes = 10

def preprocess(image, label):
    image = tf.image.resize(image, img_size)
    image = tf.cast(image, tf.float32)
    image = preprocess_input(image)
    label = tf.one_hot(label, num_classes)
    return image, label

A continuación indicamos que la función anteriormente definida se aplicará a cada imagen cuando sea necesario. Para ello utilizamos la función [map()](https://www.tensorflow.org/api_docs/python/tf/data/Dataset#map) del objeto [Dataset](https://www.tensorflow.org/api_docs/python/tf/data/Dataset) y le pasamos como parámetro la función de preproceso que queremos que sea aplique a cada muestra del conjunto de datos.

In [ ]:
train_data = train_data.map(preprocess) #usamos la funcion de preproceso como maping, como vamos cogiendo omagenes se procesan y se envian a la gpu
test_data = test_data.map(preprocess) #el cifar 10 no se puede hacer online pq no cabe en memoria hemos de procesar las imagenes en tiempo de ejecucion

Las funciones [take()](https://www.tensorflow.org/api_docs/python/tf/data/Dataset#take) y [skip()](https://www.tensorflow.org/api_docs/python/tf/data/Dataset#skip) combinadas permiten definir los conjuntos de entrenamiento y validación como nuevos Datasets.

In [ ]:
train_size = int(0.8 * train_size) #coger el 80% de datos de entrenamiento
train_dataset = train_data.take(train_size)
val_dataset = train_data.skip(train_size)#con el skip cogemos a partir del 80% para el test
test_dataset = test_data

print(len(train_dataset),len(val_dataset))

40000 10000


### Carga del modelo pre-entrenado

Seguidamente procedemos con la carga del modelo Inception V3 con los pesos resultantes de entrenarlo con la base de datos Imagenet, pero no queremos que el modelo incluya la capa de salida (include_top=False) que por defecto es una softmax de 1000 clases.

In [ ]:
from keras.applications.inception_v3 import InceptionV3 #cargar el modelo
                 #tamaño con 3 canales de color  quitamos la capa de salida, la capa densa # carga los datos de imagenet
model = InceptionV3(input_shape=img_size + (3,),include_top=False, weights='imagenet')
model.summary()

87910968/87910968 [==============================] - 0s 0us/step
Model: "inception_v3"
__________________________________________________________________________________________________
 Layer (type)                Output Shape                 Param #   Connected to                  
 input_1 (InputLayer)        [(None, 299, 299, 3)]        0         []                            
                                                                                                  
 conv2d (Conv2D)             (None, 149, 149, 32)         864       ['input_1[0][0]']             
                                                                                                  
 batch_normalization (Batch  (None, 149, 149, 32)         96        ['conv2d[0][0]']              
 Normalization)                                                                                   
                                                                                                  
 activation (Activatio

### Preparación del modelo pre-entrenado

Vamos a preparar la red Inception V3 para ser entrenada (fine-tuning) con CIFAR-10. Dado el número de parámetros de este modelo (21M), nos limitaremos a utilizarlo con los valores por defecto y añadiremos una capa GlobalAveragePooling + MLP seguida de una softmax de 10 neuronas (10 clases) acorde a CIFAR-10 que sí que entrenaremos.

In [ ]:
from keras.layers import GlobalAveragePooling2D, Dense, Dropout
from keras.models import Model
# bucle para que no entrene las capas del modelo y no toque los pesos
for layer in model.layers:
    layer.trainable = False
#añadimos capas al modelo
x = GlobalAveragePooling2D()(model.output)
#añadimos capa densa
x = Dense(1024, activation='relu')(x)
#capa dropout
x = Dropout(0.5)(x)
#finalmente capa densa con una softmax para clasificacion
output = Dense(num_classes, activation='softmax')(x)
#construimos modelo de la capa original y salida la que hemos creado, solo entrenamos la parte nueva
model = Model(inputs=model.input, outputs=output)

model.summary()

Model: "model"
__________________________________________________________________________________________________
 Layer (type)                Output Shape                 Param #   Connected to                  
 input_1 (InputLayer)        [(None, 299, 299, 3)]        0         []                            
                                                                                                  
 conv2d (Conv2D)             (None, 149, 149, 32)         864       ['input_1[0][0]']             
                                                                                                  
 batch_normalization (Batch  (None, 149, 149, 32)         96        ['conv2d[0][0]']              
 Normalization)                                                                                   
                                                                                                  
 activation (Activation)     (None, 149, 149, 32)         0         ['batch_normalization[0][0

Compilamos el modelo con los mismos parámetros que en sesiones anteriores.

In [ ]:
from keras.optimizers import Adam

opt=Adam(learning_rate=0.001)
model.compile(loss='categorical_crossentropy',
            optimizer=opt,
            metrics=['accuracy'])

Entrenamos el modelo utilizando los conjuntos de datos organizado en batches y que son cargados en memoria dinámicamente.

In [ ]:
from keras.callbacks import ReduceLROnPlateau, ModelCheckpoint
from keras.models import load_model

reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=0.2, patience=2, min_lr=0.00001)
checkpoint = ModelCheckpoint(filepath='best_model.h5', monitor='val_accuracy', save_best_only=True, verbose=1)

epochs=4
batch_size=32
train_dataset_batched = train_dataset.batch(batch_size).prefetch(tf.data.AUTOTUNE)# el prefetch hace como un pipeline para mientras uno se ejecuta el otro se va preparando
val_dataset_batched = val_dataset.batch(batch_size).prefetch(tf.data.AUTOTUNE)
history = model.fit(train_dataset_batched,
                    epochs=epochs,
                    verbose=1,
                    validation_data=val_dataset_batched,
                    callbacks=[reduce_lr,checkpoint])

Epoch 1/4
1250/1250 [==============================] - ETA: 0s - loss: 0.6049 - accuracy: 0.7996
Epoch 1: val_accuracy improved from -inf to 0.86060, saving model to best_model.h5


/usr/local/lib/python3.10/dist-packages/keras/src/engine/training.py:3079: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


1250/1250 [==============================] - 272s 202ms/step - loss: 0.6049 - accuracy: 0.7996 - val_loss: 0.4036 - val_accuracy: 0.8606 - lr: 0.0010
Epoch 2/4
1250/1250 [==============================] - ETA: 0s - loss: 0.4747 - accuracy: 0.8387
Epoch 2: val_accuracy improved from 0.86060 to 0.86540, saving model to best_model.h5
1250/1250 [==============================] - 262s 210ms/step - loss: 0.4747 - accuracy: 0.8387 - val_loss: 0.4003 - val_accuracy: 0.8654 - lr: 0.0010
Epoch 3/4
1250/1250 [==============================] - ETA: 0s - loss: 0.4409 - accuracy: 0.8502
Epoch 3: val_accuracy did not improve from 0.86540
1250/1250 [==============================] - 255s 204ms/step - loss: 0.4409 - accuracy: 0.8502 - val_loss: 0.4116 - val_accuracy: 0.8630 - lr: 0.0010
Epoch 4/4
1250/1250 [==============================] - ETA: 0s - loss: 0.4157 - accuracy: 0.8567
Epoch 4: val_accuracy improved from 0.86540 to 0.87090, saving model to best_model.h5
1250/1250 [=========================

### Cargar el mejor modelo y evaluarlo con el test set

In [ ]:
model = load_model('best_model.h5')
test_dataset_batched = test_dataset.batch(batch_size).prefetch(tf.data.AUTOTUNE)
score = model.evaluate(test_dataset_batched, verbose=0)
print(f'Test loss: {score[0]*100:.2f}')
print(f'Test accuracy: {score[1]*100:.2f}')

Test loss: 39.95
Test accuracy: 86.82


### Aumento de datos

En las sesiones anteriores, hemos utilizado la función [ImageDataGenerator](https://www.tensorflow.org/api_docs/python/tf/keras/preprocessing/image/ImageDataGenerator) para realizar el aumento de datos. Sin embargo, esta función no se recomienda para los nuevos desarrollos de código por estar obsoleta, y en su lugar se deben utilizar las [capas de preproceso](https://www.tensorflow.org/guide/keras/preprocessing_layers). Más concretamente, utilizaremos algunas de las [capas de preproceso de aumento de datos para imágenes](https://www.tensorflow.org/guide/keras/preprocessing_layers#image_data_augmentation).

Añadiremos estas capas de aumento de datos antes del modelo Inception V3.

In [ ]:
from keras.applications.inception_v3 import InceptionV3
from keras.layers import Input, GlobalAveragePooling2D, Dense, Dropout, RandomRotation, RandomTranslation, RandomZoom
from keras.models import Model

input_layer = Input(shape=img_size + (3,))
#capa de data aumentation, rotaciones, translaciones, etc.
x = RandomRotation(factor=0.1, fill_mode='nearest')(input_layer)
x = RandomTranslation(height_factor=0.1, width_factor=0.1, fill_mode='nearest')(x)
x = RandomZoom(height_factor=0.2, fill_mode='nearest')(x)


inception_model = InceptionV3(input_shape=img_size + (3,),include_top=False, weights='imagenet')

for layer in inception_model.layers:
    layer.trainable = False

x = inception_model(x)

x = GlobalAveragePooling2D()(x)
x = Dense(1024, activation='relu')(x)
x = Dropout(0.5)(x)
output = Dense(num_classes, activation='softmax')(x)

aug_model = Model(inputs=input_layer, outputs=output)

aug_model.summary()

Model: "model_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_2 (InputLayer)        [(None, 299, 299, 3)]     0         
                                                                 
 random_rotation (RandomRot  (None, 299, 299, 3)       0         
 ation)                                                          
                                                                 
 random_translation (Random  (None, 299, 299, 3)       0         
 Translation)                                                    
                                                                 
 random_zoom (RandomZoom)    (None, 299, 299, 3)       0         
                                                                 
 inception_v3 (Functional)   (None, 8, 8, 2048)        21802784  
                                                                 
 global_average_pooling2d_1  (None, 2048)              0   

In [ ]:
from keras.optimizers import Adam

opt=Adam(learning_rate=0.001)
aug_model.compile(loss='categorical_crossentropy',
            optimizer=opt,
            metrics=['accuracy'])

In [ ]:
from keras.callbacks import ReduceLROnPlateau, ModelCheckpoint
from keras.models import load_model

reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=0.2, patience=2, min_lr=0.00001)
checkpoint = ModelCheckpoint(filepath='best_model.h5', monitor='val_accuracy', save_best_only=True, verbose=1)

epochs=4
batch_size=32
train_dataset_batched = train_dataset.batch(batch_size).prefetch(tf.data.AUTOTUNE)
val_dataset_batched = val_dataset.batch(batch_size).prefetch(tf.data.AUTOTUNE)
history = aug_model.fit(train_dataset_batched,
                    epochs=epochs,
                    verbose=1,
                    validation_data=val_dataset_batched,
                    callbacks=[reduce_lr,checkpoint])

Epoch 1/4
1250/1250 [==============================] - ETA: 0s - loss: 0.7908 - accuracy: 0.7234
Epoch 1: val_accuracy improved from -inf to 0.85160, saving model to best_model.h5
1250/1250 [==============================] - 255s 204ms/step - loss: 0.7908 - accuracy: 0.7234 - val_loss: 0.4491 - val_accuracy: 0.8516 - lr: 0.0010
Epoch 2/4
1250/1250 [==============================] - ETA: 0s - loss: 0.7604 - accuracy: 0.7357
Epoch 2: val_accuracy improved from 0.85160 to 0.85670, saving model to best_model.h5
1250/1250 [==============================] - 261s 209ms/step - loss: 0.7604 - accuracy: 0.7357 - val_loss: 0.4207 - val_accuracy: 0.8567 - lr: 0.0010
Epoch 3/4
1250/1250 [==============================] - ETA: 0s - loss: 0.7454 - accuracy: 0.7423
Epoch 3: val_accuracy did not improve from 0.85670
1250/1250 [==============================] - 256s 205ms/step - loss: 0.7454 - accuracy: 0.7423 - val_loss: 0.4231 - val_accuracy: 0.8547 - lr: 0.0010
Epoch 4/4
1250/1250 [==================

In [ ]:
aug_model = load_model('best_model.h5')
test_dataset_batched = test_dataset.batch(batch_size).prefetch(tf.data.AUTOTUNE)
score = aug_model.evaluate(test_dataset_batched, verbose=0)
print(f'Test loss: {score[0]*100:.2f}')
print(f'Test accuracy: {score[1]*100:.2f}')

## Ejercicio:

Como habrás podido comprobar, el fine-tuning de modelos pre-entrenados a una tarea en concreto abre un amplio abanico de posibilidades para mejorar la tasa de acierto en esa tarea. Prueba a realizar fine-tuning de [otros modelos pre-entrenados para la clasificación de imágenes](https://keras.io/api/applications/) y supera la tasa de acierto del 90%.